## Introduction

This is the submission code of "RSNA Screening Mammography Breast Cancer Detection"

Reference:
https://www.kaggle.com/code/christofhenkel/se-resnext50-full-gpu-decoding

https://www.kaggle.com/code/markwijkhuizen/rsna-efficientnetv2-training-tensorflow-tpu

https://www.kaggle.com/code/markwijkhuizen/rsna-cropped-tfrecords-768x1344-dataset





## Requirements

We start with installing pip requirements.

In [ ]:
!pip install -q timm==0.6.5 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q albumentations==1.2.1 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q pylibjpeg-libjpeg==1.3.1 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
# !pip install -q pydicom==2.0.0 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q python-gdcm==3.0.20 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q dicomsdl==0.109.1 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements

In [ ]:
# install the latest DALI packaging which we will use for GPU decoding
!pip install -q /kaggle/input/nvidia-dali-nightly-cuda110-1230dev/nvidia_dali_nightly_cuda110-1.23.0.dev20230203-7187866-py3-none-manylinux2014_x86_64.whl

In [ ]:
# Install Keras CV Attention Model Pip Package for ConvNextV2 Models
!pip install --no-deps /kaggle/input/keras-cv-attention-models/keras_cv_attention_models-1.3.9-py3-none-any.whl

In [ ]:
#we need to patch DALI for Int16 support
from nvidia.dali.backend import TensorGPU, TensorListGPU
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
from nvidia.dali import types
from nvidia.dali.plugin.base_iterator import _DaliBaseIterator
from nvidia.dali.plugin.base_iterator import LastBatchPolicy
import torch
import torch.utils.dlpack as torch_dlpack
import ctypes
import numpy as np
import torch.nn.functional as F
import pydicom

to_torch_type = {
    types.DALIDataType.FLOAT:   torch.float32,
    types.DALIDataType.FLOAT64: torch.float64,
    types.DALIDataType.FLOAT16: torch.float16,
    types.DALIDataType.UINT8:   torch.uint8,
    types.DALIDataType.INT8:    torch.int8,
    types.DALIDataType.UINT16:  torch.int16,
    types.DALIDataType.INT16:   torch.int16,
    types.DALIDataType.INT32:   torch.int32,
    types.DALIDataType.INT64:   torch.int64
}


def feed_ndarray(dali_tensor, arr, cuda_stream=None):
    """
    Copy contents of DALI tensor to PyTorch's Tensor.

    Parameters
    ----------
    `dali_tensor` : nvidia.dali.backend.TensorCPU or nvidia.dali.backend.TensorGPU
                    Tensor from which to copy
    `arr` : torch.Tensor
            Destination of the copy
    `cuda_stream` : torch.cuda.Stream, cudaStream_t or any value that can be cast to cudaStream_t.
                    CUDA stream to be used for the copy
                    (if not provided, an internal user stream will be selected)
                    In most cases, using pytorch's current stream is expected (for example,
                    if we are copying to a tensor allocated with torch.zeros(...))
    """
    dali_type = to_torch_type[dali_tensor.dtype]

    assert dali_type == arr.dtype, ("The element type of DALI Tensor/TensorList"
                                    " doesn't match the element type of the target PyTorch Tensor: "
                                    "{} vs {}".format(dali_type, arr.dtype))
    assert dali_tensor.shape() == list(arr.size()), \
        ("Shapes do not match: DALI tensor has size {0}, but PyTorch Tensor has size {1}".
            format(dali_tensor.shape(), list(arr.size())))
    cuda_stream = types._raw_cuda_stream(cuda_stream)

    # turn raw int to a c void pointer
    c_type_pointer = ctypes.c_void_p(arr.data_ptr())
    if isinstance(dali_tensor, (TensorGPU, TensorListGPU)):
        stream = None if cuda_stream is None else ctypes.c_void_p(cuda_stream)
        dali_tensor.copy_to_external(c_type_pointer, stream, non_blocking=True)
    else:
        dali_tensor.copy_to_external(c_type_pointer)
    return arr


In [ ]:
# Params
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import multiprocessing as mp

COMP_FOLDER = '/kaggle/input/rsna-breast-cancer-detection/'
DATA_FOLDER = COMP_FOLDER + 'test_images/'

sample_submission = pd.read_csv(COMP_FOLDER + 'sample_submission.csv')

PUBLIC_RUN = len(sample_submission) == 2

N_CORES = mp.cpu_count()
MIXED_PRECISION = False
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

RAM_CHECK = False
DEBUG = False

test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
test_df['cancer'] = 0 #dummy value


if PUBLIC_RUN is False:
    RAM_CHECK = False
    DEBUG = False

if RAM_CHECK is True:
    test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')[:50]
    
    test_df = test_df[test_df.image_id!=1942326353].reset_index(drop=True)
    print(test_df.shape)
    
    patient_filter = list(sorted((set(test_df.patient_id.unique()))))
    test_df = test_df[test_df.patient_id.isin(patient_filter)]
    DATA_FOLDER = DATA_FOLDER.replace('test','train')

if DEBUG is True:
    test_df = test_df.head(500)

test_df

In [ ]:
print(f'Len df : {len(test_df)}')
test_df['patient_id'].nunique()
test_df["fns"] = test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.dcm'

In [ ]:
# we define the function for GPU-based decoding using DALI and processing the dicom images
import dicomsdl
import pydicom
from pydicom.filebase import DicomBytesIO

import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali import pipeline_def
from nvidia.dali.types import DALIDataType


def convert_dicom_to_jpg(file, save_folder=""):
    patient = file.split('/')[-2]
    image = file.split('/')[-1][:-4]
    dcmfile = pydicom.dcmread(file)

    if dcmfile.file_meta.TransferSyntaxUID == '1.2.840.10008.1.2.4.90':
        with open(file, 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\x00\x00\x00\x0C")  #<---- the jpeg2000 header info we're looking for
        hackedbitstream = bytearray()
        hackedbitstream.extend(ds.PixelData[offset:])
        with open(save_folder + f"{patient}_{image}.jpg", "wb") as binary_file:
            binary_file.write(hackedbitstream)
            
    if dcmfile.file_meta.TransferSyntaxUID == '1.2.840.10008.1.2.4.70':
        with open(file, 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\xff\xd8\xff\xe0")  #<---- the jpeg lossless header info we're looking for
        hackedbitstream = bytearray()
        hackedbitstream.extend(ds.PixelData[offset:])
        with open(save_folder + f"{patient}_{image}.jpg", "wb") as binary_file:
            binary_file.write(hackedbitstream)

            
@pipeline_def
def jpg_decode_pipeline(jpgfiles):
    jpegs, _ = fn.readers.file(files=jpgfiles)
    images = fn.experimental.decoders.image(jpegs, device = 'mixed', output_type = types.ANY_DATA, dtype = DALIDataType.UINT16)
    return images

def parse_window_element(elem):
    if type(elem) == list:
        return float(elem[0])
    if type(elem) == str:
        return float(elem)
    if type(elem) == float:
        return elem
    if type(elem) == pydicom.dataelem.DataElement:
        try:
            return float(elem[0])
        except:
            return float(elem.value)
    return None

def linear_window(data, center, width):
    lower, upper = center - width // 2, center + width // 2
    data = torch.clamp(data, min = lower, max = upper)
    return data 

def process_dicom(img, dicom):
    try:
        invert = getattr(dicom, "PhotometricInterpretation", None) == "MONOCHROME1"
    except:
        invert = False
        
    center = parse_window_element(dicom["WindowCenter"]) 
    width = parse_window_element(dicom["WindowWidth"])
        
    if (center is not None) & (width is not None):
        img = linear_window(img, center, width)

    img = (img - img.min()) / (img.max() - img.min())
    if invert:
        img = 1 - img
    return img

In [ ]:
from types import SimpleNamespace
from typing import Any, Dict

cfg = SimpleNamespace(**{})
cfg.img_size = 1024
cfg.backbone = 'seresnext50_32x4d'
cfg.pretrained=False
cfg.in_channels = 1
cfg.classes = ['cancer']
cfg.batch_size = 8
cfg.data_folder = "/tmp/output/"
cfg.val_aug = A.CenterCrop(always_apply=False, p=1.0, height=cfg.img_size, width=cfg.img_size)
cfg.device = DEVICE

In [ ]:
# We will process the dicoms in chunks so the disk space does not become an issue.
import os
SAVE_SIZE = int(cfg.img_size * 1.125)
SAVE_FOLDER = cfg.data_folder
os.makedirs(SAVE_FOLDER, exist_ok=True)
N_CHUNKS = len(test_df["fns"]) // 2000 if len(test_df["fns"]) > 2000 else 1
CHUNKS = [(len(test_df["fns"]) / N_CHUNKS * k, len(test_df["fns"]) / N_CHUNKS * (k + 1)) for k in range(N_CHUNKS)]
CHUNKS = np.array(CHUNKS).astype(int)
JPG_FOLDER = "/tmp/jpg/"

# model 1

Loading libraries

In [ ]:
# import library needed for model1
import torch
from torch import nn
import torch.nn.functional as F
from typing import Any
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader
import timm
import cv2
import numpy as np
import gc
import glob
import tqdm
import pandas as pd
from types import SimpleNamespace
import albumentations as A
import multiprocessing as mp

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
N_CORES = mp.cpu_count()

In [ ]:
cfg = SimpleNamespace(**{})
cfg.img_size = 1024
cfg.backbone = 'seresnext50_32x4d'
cfg.pretrained=False
cfg.in_channels = 1
cfg.classes = ['cancer']
cfg.batch_size = 8
cfg.data_folder = "/tmp/output/"
cfg.val_aug = A.CenterCrop(always_apply=False, p=1.0, height=cfg.img_size, width=cfg.img_size)
cfg.device = DEVICE

In [ ]:
# We will process the dicoms in chunks so the disk space does not become an issue.
SAVE_SIZE = int(cfg.img_size * 1.125)
SAVE_FOLDER = cfg.data_folder
os.makedirs(SAVE_FOLDER, exist_ok=True)
N_CHUNKS = len(test_df["fns"]) // 2000 if len(test_df["fns"]) > 2000 else 1
CHUNKS = [(len(test_df["fns"]) / N_CHUNKS * k, len(test_df["fns"]) / N_CHUNKS * (k + 1)) for k in range(N_CHUNKS)]
CHUNKS = np.array(CHUNKS).astype(int)
JPG_FOLDER = "/tmp/jpg/"

In [ ]:
import shutil 
from joblib import Parallel, delayed
for ttt, chunk in enumerate(CHUNKS):
    print(f'chunk {ttt} of {len(CHUNKS)} chunks')
    os.makedirs(JPG_FOLDER, exist_ok=True)

    _ = Parallel(n_jobs=2)(
        delayed(convert_dicom_to_jpg)(f'{DATA_FOLDER}/{img}', save_folder=JPG_FOLDER)
        for img in test_df["fns"].tolist()[chunk[0]: chunk[1]]
    )
    
    jpgfiles = glob.glob(JPG_FOLDER + "*.jpg")


    pipe = jpg_decode_pipeline(jpgfiles, batch_size=1, num_threads=2, device_id=0)
    pipe.build()

    for i, f in enumerate(tqdm(jpgfiles)):
        
        patient, dicom_id = f.split('/')[-1][:-4].split('_')
        dicom = pydicom.dcmread(DATA_FOLDER + f"/{patient}/{dicom_id}.dcm")
        try:
            out = pipe.run()
            # Dali -> Torch
            img = out[0][0]
            img_torch = torch.empty(img.shape(), dtype=torch.int16, device="cuda")
            feed_ndarray(img, img_torch, cuda_stream=torch.cuda.current_stream(device=0))
            img = img_torch.float()

            


            #apply dicom preprocessing
            img = process_dicom(img, dicom)

            #resize the torch image
            img = F.interpolate(img.view(1, 1, img.size(0), img.size(1)), (SAVE_SIZE, SAVE_SIZE), mode="bilinear")[0, 0]

            img = (img * 255).clip(0,255).to(torch.uint8).cpu().numpy()
            out_file_name = SAVE_FOLDER + f"{patient}_{dicom_id}.png"
            cv2.imwrite(out_file_name, img)
    
        except Exception as e:
            print(i, e)
            pipe = jpg_decode_pipeline(jpgfiles[i+1:], batch_size=1, num_threads=2, device_id=0)
            pipe.build()
            continue

    shutil.rmtree(JPG_FOLDER)
print(f'DALI Raw image load complete')

In [ ]:
fns = glob.glob(f'{SAVE_FOLDER}/*.png')
n_saved = len(fns)
print(f'Image on disk count : {n_saved}')

In [ ]:
gpu_processed_files = [fn.split('/')[-1].replace('_','/').replace('png','dcm') for fn in fns]
to_process = [f for f in test_df["fns"].values if f not in gpu_processed_files]
len(gpu_processed_files), len(to_process)

In [ ]:
def process(f, save_folder=""):
    patient = f.split('/')[-2]
    dicom_id = f.split('/')[-1][:-4]
    
    dicom = dicomsdl.open(f)
    img = dicom.pixelData()
    img = torch.from_numpy(img)
    img = process_dicom(img, dicom)
    
    img = F.interpolate(img.view(1, 1, img.size(0), img.size(1)), (SAVE_SIZE, SAVE_SIZE), mode="bilinear")[0, 0]

    img = (img * 255).clip(0,255).to(torch.uint8).cpu().numpy()
    out_file_name = SAVE_FOLDER + f"{patient}_{dicom_id}.png"
    cv2.imwrite(out_file_name, img)
    return out_file_name

In [ ]:
cpu_processed_filenames = Parallel(n_jobs=2)(
    delayed(process)(f'{DATA_FOLDER}/{img}', save_folder=SAVE_FOLDER)
    for img in tqdm(to_process)
)
cpu_processed_filenames = [f for f in cpu_processed_filenames if f]
print(f'CPU Raw image load complete with {len(cpu_processed_filenames)} loaded')

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
n_saved = len(glob.glob(f'{SAVE_FOLDER}/*.png'))
print(f'Image on disk count : {n_saved}')

In [ ]:
assert n_saved == len(test_df)

Data Preparing

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast


def batch_to_device(batch, device):
    batch_dict = {key: batch[key].to(device) for key in batch}
    return batch_dict

class CustomDataset(Dataset):
    def __init__(self, df, cfg, aug):

        self.cfg = cfg
        self.df = df.copy()
        self.df = self.df[self.df['image_id'].astype(str) != '1942326353']
        self.labels = self.df[self.cfg.classes].values
        self.df["fns"] = self.df['patient_id'].astype(str) + '_' + self.df['image_id'].astype(str) + '.png'
        self.fns = self.df["fns"].astype(str).values
        self.aug = aug
        self.data_folder = cfg.data_folder

    def __getitem__(self, idx):

        label = self.labels[idx]
        img = self.load_one(idx)

        if self.aug:
            img = self.augment(img)

        img = self.normalize_img(img)
        torch_img = torch.tensor(img).float().permute(2,0,1)
        
        feature_dict = {
            "input": torch_img,
            "target": torch.tensor(label),
        }
        return feature_dict

    def __len__(self):
        return len(self.fns)

    def load_one(self, idx):
        path = self.data_folder + self.fns[idx]
        try:
            img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
            shape = img.shape
            if len(img.shape) == 2:
                img = img[:,:,None]
        except Exception as e:
            print(e)
        return img

    def augment(self, img):
        img = img.astype(np.float32)
        transformed = self.aug(image = img)
        trans_img = transformed["image"]
        return trans_img

    def normalize_img(self, img):
        img = img / 255
        return img

GeM pooling

In [ ]:
def gem(x, p = 3, eps = 1e-6):
    return F.avg_pool2d(x.clamp(min = eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    def __init__(self, p = 3, eps = 1e-6, p_trainable = False):
        super(GeM, self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1) * p)
        else:
            self.p = p

        self.eps = eps
    
    def forward(self, x):
        ret = gem (x, p=self.p, eps=self.eps)
        return ret

Constructing Network

In [ ]:
class Net(nn.Module):
    def __init__(self, cfg: Any):
        super(Net, self).__init__()
        self.cfg = cfg
        self.n_class = len(cfg.classes)
        self.backbone = timm.create_model(cfg.backbone,
                                          pretrained = cfg.pretrained,
                                          num_classes = 0,
                                          global_pool = "",
                                          in_chans = self.cfg.in_channels)
        
        backbone_out = self.backbone.feature_info[-1]['num_chs']
        self.global_pool = GeM(p_trainable = False)
        self.head = torch.nn.Linear(backbone_out, self.n_class)
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, batch):
        x = batch['input']
        x = self.backbone(x)
        x = self.global_pool(x)
        x = x[:, :, 0, 0]

        logits = self.head(x)

        outputs = {}

        if self.training:
            loss = self.loss_fn(logits, batch["target"].float())
            outputs['loss'] = loss

        else:
            outputs['logits'] = logits
        
        return outputs

Training Network

In [ ]:
def get_dl(test_df, cfg):
    test_ds = CustomDataset(test_df, cfg, cfg.val_aug)
    test_dl = DataLoader(test_ds, shuffle = False, batch_size = cfg.batch_size, num_workers = N_CORES, pin_memory = True)

    return test_dl, batch_to_device


def get_state_dict(sd_fp):
    sd = torch.load(sd_fp, map_location="cpu")
    sd = {k.replace("module.", ""): v for k, v in sd.items()}
    return sd

def get_nets(cfg, state_dicts):
    nets = []

    for i, state_dict in enumerate(state_dicts):
        net = Net(cfg).eval.to(DEVICE)
        print("loading dict")
        sd = get_state_dict(state_dict)
        net.load_state_dict(sd, strict = True)
        nets += [net]
        del sd
        gc.collect()
    
    return nets

sub_dl, batch_to_device = get_dl()
state_dicts = sorted(glob.glob('/kaggle/input/rsna-seresnext50-5fold/check*.pth'))

nets = get_nets(cfg, state_dicts)
print(f'Dataloader length : {len(sub_dl.dataset)}')

with torch.inference_mode():

    preds = [[] for i in range(len(nets))]
    for batch in tqdm(sub_dl):
        batch = batch_to_device(batch, cfg.device)
        for i, net in enumerate(nets):
            logits = net(batch)['logits'].sigmoid().float().detach().cpu().numpy()
            preds[i] += [logits]

preds = np.array([np.concatenate(p, axis = 0) for p in preds])
preds = preds.mean(0) #average fold predictions
preds = preds[:,0]

patient_id = sub_dl.dataset.df['patient_id'].values
laterality = sub_dl.dataset.df['laterality'].values

prediction_id = [f'{i}_{j}' for i,j in  zip(patient_id, laterality)]

pred_df = pd.DataFrame({'prediction_id': prediction_id, 'cancer_raw': preds})

#aggregate by prediction_id , i.e. by patient_laterality
sub = pred_df.groupby('prediction_id')[['cancer_raw']].agg('mean')
sub[['cancer_raw']].to_csv('sub1.csv')

### Model2 

In [ ]:
import tensorflow as tf
from keras_cv_attention_models import convnext

In [ ]:
TARGET_HEIGHT = 1152
TARGET_WIDTH  = 1152
N_CHANNELS = 1
INPUT_SHAPE = (TARGET_HEIGHT, TARGET_WIDTH, N_CHANNELS)
TARGET_HEIGHT_WIDTH_RATIO = 1
THRESHOLD_BEST = 0.50

CLAHE = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (32, 32))

CROP_IMAGE = True
APPLY_CLAHE = False
APPLY_EQ_HIST = False

IMAGE_FORMAT = 'jpg'

In [ ]:
def normalize(image):
    # Repeat channels to create 3 channel images required by pretrained ConvNextV2 models 
    image = tf.repeat(image, repeats = 3, axis = 3)
    # Cast to float 32
    image = tf.cast(image, tf.float32)
    # Normalize with respect to ImageNet mean/std
    image = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='torch')

    return image


def get_model(weightpath):
    # Inputs, note the names are equal to the dictionary keys in the dataset
    image = tf.keras.layers.Input(INPUT_SHAPE, TARGET_WIDTH, N_CHANNELS)

    # Normalize Input
    image_norm = normalize(image)

    # CNN Feature Maps
    x = convnext.ConvNeXtV2Tiny(
        input_shape = (TARGET_HEIGHT, TARGET_WIDTH, 3),
        pretrained  = None,
        num_classes = 0,
    )(image_norm)

    # Average Pooling BxHxWxC -> BxC
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # Dropout to prevent Overfitting
    x = tf.keras.layers.Dropout(0.30)(x)
    # Output value between [0, 1] using Sigmoid function
    outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    # Define model with inputs and outputs
    model = tf.keras.models.Model(inputs = image, outputs = outputs)

    # Load pretrained Model Weights
    model.load_weights(weightpath)
#     model.load_weights('/kaggle/input/rsna-efficientnetv2-training-tensorflow-tpu-ds/model.h5')

    # Set model non-trainable
    model.trainable = False

    # Compile model
    model.compile()

    return model

In [ ]:
# Pretrained File Path: '/kaggle/input/sartorius-training-dataset/model.h5'
tf.keras.backend.clear_session()
# enable XLA optmizations
tf.config.optimizer.set_jit(True)

# model = get_model()
modellist = []
from keras.models import load_model

for ii in [2,3,4]:
    model = load_model(f'/kaggle/input/rsnamodel/fold{ii}.h5')
    modellist.append(model)


model.summary()

Data Loading

In [ ]:
from typing import (
    Dict, Optional, Union, List, Tuple, TYPE_CHECKING, cast, Iterable,
    ByteString
)
from pydicom.valuerep import VR

# Binarize the image at the threshold
def _binarize(img, threshold):
    return (img > threshold).astype(np.uint8)

# Get contour points of the breast
def _extract_contour(bin_img):
    contours, _ = cv2.findContours(
        bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)
    return contour


# Set to background pixels of the image to zero
def _erase_background(img, contour):
    mask = np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
    output = cv2.bitwise_and(img, mask)
    return output
    
 # Crop the useless background of the image
def img2roi(img):
    
    # Flip T0 Left/Right Orientation
    h0, w0 = img.shape
    if img[:,int(-w0 * 0.10):].sum() > img[:,:int(w0 * 0.10)].sum():
        img = np.flip(img, axis=1)
        
        
    bin_img = _binarize(img, threshold = 5)
    contour = _extract_contour(bin_img)
    img = _erase_background(img, contour)
    x1, x2 = np.min(contour[:, :, 0]), np.max(contour[:, :, 0])
    y1, y2 = np.min(contour[:, :, 1]), np.max(contour[:, :, 1])
    x1, x2 = int(0.975 * x1), int(1.025 * x2)
    y1, y2 = int(0.975 * y1), int(1.025 * y2)
    
    x_offset = x2#get_x_offset(image, debug=debug)
    offset_bottom, offset_top = y1, y2#get_y_offsets(image[:,:x_offset], debug=debug)
    # Crop Height and Width
    h_crop = offset_top - offset_bottom
    w_crop = x_offset - x1
    
    # Pad crop offsets to target aspect ratio
    # Height too large, pad x offset
    if (h_crop / w_crop) > TARGET_HEIGHT_WIDTH_RATIO:
#         print('add x')
        x_offset += int(h_crop / TARGET_HEIGHT_WIDTH_RATIO - w_crop)
    else:
#         print('add y')
        # Height too small, pad bottom/top offsets
        offset_bottom -= int(0.50 * (w_crop * TARGET_HEIGHT_WIDTH_RATIO - h_crop))
        offset_bottom_correction = max(0, -offset_bottom)
        offset_bottom += offset_bottom_correction

        offset_top += int(0.50 * (w_crop * TARGET_HEIGHT_WIDTH_RATIO - h_crop))
        offset_top += offset_bottom_correction
        
    # Crop Image
#     image = image[offset_bottom:offset_top:,:x_offset]
    
    return img[offset_bottom: offset_top, x1: x_offset]  

In [ ]:
import matplotlib.pyplot as plt
SUBMISSION_ROWS = []
# Iterate over all patient_id/laterality combinations groups
for idx, ((patient_id, laterality), g) in enumerate(tqdm(test.groupby(['patient_id', 'laterality']))):
    # Cancer target is mean of predicted cancer values
    cancer = []#0
    # Iterate over all scans in group
    for row_idx, row in g.iterrows():
        # Load Image
        image_id = row['image_id']
        image = cv2.imread(f'{SAVE_FOLDER}/{patient_id}_{image_id}.png', cv2.IMREAD_GRAYSCALE)
        image = img2roi(image)
        image = cv2.resize(image, (SAVE_SIZE,SAVE_SIZE), interpolation=cv2.INTER_AREA)
        
        
        # Show First Few Images
        if idx < 16:
            plt.figure(figsize=(5,8))
            plt.imshow(image)
            plt.show()
        
        # Expand to Batch HxW -> 1xHxWx1
        image = np.expand_dims(image, [0, 3])
        # Make Prediction
        
        for model in modellist:
            cancer.append(model.predict_on_batch(image).squeeze() / len(g) /len(modellist))
    
#         cancer += model.predict_on_batch(image).squeeze() / len(g)
        # Remove Image
#         os.remove(f'{SAVE_FOLDER}/{patient_id}_{image_id}.png')
        
    # Add Submission Row
    SUBMISSION_ROWS.append({
        'prediction_id': f'{patient_id}_{laterality}',
        'cancer_raw': np.median(cancer),
    })
    
    if np.random.rand() > 0.99:
        gc.collect()

In [ ]:
# Create DataFrame from submission rows
submission_df = pd.DataFrame(SUBMISSION_ROWS)
submission_df

In [ ]:
# binarize predictions
th = np.quantile(submission_df['cancer_raw'].values,0.97935)
# th = THRESHOLD_BEST.copy()
print('THRESHOLD_BEST:',th)

# submission_df['cancer'] = (submission_df['cancer_raw'].values > th).astype(int)

In [ ]:
submission_df[['prediction_id','cancer_raw']].to_csv('sub2.csv', index=False)

# Merge two results

In [ ]:
sub1 = pd.read_csv('sub1.csv')
sub2 = pd.read_csv('sub2.csv')
sub = pd.merge(sub1,sub2,how='left',on='prediction_id')
sub.columns = ['prediction_id','cancer_raw_1','cancer_raw_2']
sub['cancer_raw'] = sub['cancer_raw_1'].rank(pct = True) * 0.5 + sub['cancer_raw_2'].rank(pct = True) * 0.5

In [ ]:
th = np.quantile(sub['cancer_raw'].values,0.97935)
sub['cancer'] = (sub['cancer_raw'].values > th).astype(int)
sub[['prediction_id','cancer']].to_csv('submission.csv', index = False)